In [1]:
seed=123
import numpy as np
np.random.seed(seed)
import tensorflow as tf
tf.set_random_seed(seed)
import random
random.seed(seed)
import os
import sys
import time
from mrcnn.my_bowl_dataset import BowlDataset
import mrcnn.model as modellib
from mrcnn.model import log
import numpy as np
from imgaug import augmenters as iaa
import skimage.io 


#######################################################################################
## SET UP CONFIGURATION
from mrcnn.config import Config

class BowlConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = "001"

    IMAGE_RESIZE_MODE = "crop"
    # Augmentation parameters
    ASPECT_RATIO = 1.3 ## Maximum aspect ratio modification when scaling
    MIN_ENLARGE = 1.2 ## Minimum enlarging of images, note that this will be randomized
    ZOOM = 1.5 ## Maximum zoom per image, note that this will be randomized
    IMAGE_MIN_SCALE = False ## Not using this

    ROT_RANGE = 10.
    CHANNEL_SHIFT_RANGE = 15
    
    LEARNING_RATE = 0.001
    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 2

    
    # Number of classes (including background)
    NUM_CLASSES = 1 + 1 # background + nuclei

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (8 , 16, 32, 64, 128)  # anchor side in pixels
    # RPN_ANCHOR_SCALES = (4, 8 , 16, 32, 64)  # anchor side in pixels

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 600

    STEPS_PER_EPOCH = 664//IMAGES_PER_GPU
    VALIDATION_STEPS = 2//IMAGES_PER_GPU ## We are training with the whole dataset so validation is not very meaningfull, I put a two here so it is faster. We either use train loss or calculate in a separate procceses the mAP for each epoch

    # use small validation steps since the epoch is small
    # VALIDATION_STEPS = 5

    USE_MINI_MASK = True

    MAX_GT_INSTANCES = 256

    DETECTION_MAX_INSTANCES = 512

bowl_config = BowlConfig()
bowl_config.display()
#######################################################################################

# Root directory of the project
ROOT_DIR = '/attached/home/git_repos/DSB_2018/datasets/calcium'


## Change this dir to the stage 1 training data
train_dir = os.path.join(ROOT_DIR,'train')

# Get train IDs
train_ids = next(os.walk(train_dir))[1]
train_ids = [os.path.join(train_dir, train_id) for train_id in train_ids]

# Training dataset
dataset_train = BowlDataset()
dataset_train.load_bowl(train_ids)
dataset_train.prepare()

# # Validation dataset, same as training.. will use pad64 on this one
dataset_val = BowlDataset()
dataset_val.load_bowl(train_ids)
dataset_val.prepare()

Using TensorFlow backend.



Configurations:
ASPECT_RATIO                   1.3
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
CHANNEL_SHIFT_RANGE            15
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        512
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  512
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                False
IMAGE_RESIZE_MODE              crop
IMAGE_SHAPE                    [512 512   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE     

### all this is testing / inspection

In [5]:
import matplotlib.pyplot as plt

In [ ]:
for img_idx in range(13):
    underlay = dataset_train.load_image(img_idx)
    overlay = np.sum(dataset_train.load_mask(img_idx)[0], axis=2)
    mixture = underlay.copy()
    mixture[:, :, 2] = overlay*255
    
    fig = plt.figure(figsize=(12, 4.5))
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.imshow(underlay)
    ax2 = fig.add_subplot(1, 2, 2)
    ax2.imshow(mixture)
    fig.suptitle(dataset_train.image_info[img_idx]['path'], color='w')

In [6]:
dataset_train.load_image(0).shape

(229, 229, 3)

In [7]:
dataset_train.load_image(0).dtype

dtype('uint8')

In [8]:
dataset_train.load_image(3).shape

(198, 203, 3)

In [9]:
dataset_train.load_mask(0)

(array([[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        ...,
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],

In [10]:
len(dataset_train.load_mask(0))

2

In [11]:
dataset_train.load_mask(0)[0].max()

1

In [12]:
dataset_train.load_mask(0)[1].shape

(85,)

In [13]:
dataset_train.load_mask(0)[0].shape

(229, 229, 85)

In [14]:
from PIL import Image

In [15]:
image_arr = np.array(Image.open('datasets/nucleus/stage1_train/00071198d059ba7f5914a526d124d28e6d010c92466da21d4a04cd5413362552/masks/07a9bf1d7594af2763c86e93f05d22c4d5181353c6d3ab30a345b908ffe5aadc.png'))

In [17]:
image_arr.dtype

dtype('uint8')

### so image input is 3-channel uint8 png at between 64x64x3 and 256x256x3

### label input is separate binary 2D pngs for each neuron

In [ ]:
### so directory structure is the following
top folder
|
    each patch (or subpatch)
    |
        images
        |
            3-chan-image-name.png
        masks
        |
            neuron-0-mask.png
            neuron-1-mask.png
            .
            .
            .

### back to training

In [2]:
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
## https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5
COCO_MODEL_PATH = '/attached/home/git_repos/DSB_2018/datasets/nucleus/deepretina_final.h5'

model = modellib.MaskRCNN(mode="training", config=bowl_config,
                          model_dir=MODEL_DIR)

model.load_weights(COCO_MODEL_PATH, by_name=True,
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                            "mrcnn_bbox", "mrcnn_mask"])

import time
start_time = time.time()

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
augmentation=False

model.train(dataset_train, dataset_val, 
            learning_rate=bowl_config.LEARNING_RATE,
            epochs=30,
            augmentation=augmentation,
            augment=True,
            layers="all")


model.train(dataset_train, dataset_val, 
            learning_rate=bowl_config.LEARNING_RATE/10,
            epochs=50,
            augmentation=augmentation,
            augment=True,
            layers="all")

model.train(dataset_train, dataset_val, 
            learning_rate=bowl_config.LEARNING_RATE/30,
            epochs=75,
            augmentation=augmentation,
            augment=True,
            layers="all")


end_time = time.time()
ellapsed_time = (end_time-start_time)/3600

print(model.log_dir)
model_path = os.path.join(model.log_dir, 'final.h5')
model.keras_model.save_weights(model_path)


Starting at epoch 0. LR=0.001

Checkpoint Path: /attached/home/git_repos/DSB_2018/datasets/calcium/logs/00120190805T1913/mask_rcnn_001_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
res3a_branch2b     

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/30
332/332 [==============================] - 295s 888ms/step - loss: 1.6924 - rpn_class_loss: 0.1466 - rpn_bbox_loss: 0.5502 - mrcnn_class_loss: 0.1383 - mrcnn_bbox_loss: 0.4233 - mrcnn_mask_loss: 0.4339 - val_loss: 1.4092 - val_rpn_class_loss: 0.0950 - val_rpn_bbox_loss: 0.3918 - val_mrcnn_class_loss: 0.2546 - val_mrcnn_bbox_loss: 0.3152 - val_mrcnn_mask_loss: 0.3527
Epoch 2/30
332/332 [==============================] - 200s 601ms/step - loss: 1.5246 - rpn_class_loss: 0.1077 - rpn_bbox_loss: 0.4940 - mrcnn_class_loss: 0.1427 - mrcnn_bbox_loss: 0.3781 - mrcnn_mask_loss: 0.4021 - val_loss: 1.3991 - val_rpn_class_loss: 0.0983 - val_rpn_bbox_loss: 0.3738 - val_mrcnn_class_loss: 0.2273 - val_mrcnn_bbox_loss: 0.3337 - val_mrcnn_mask_loss: 0.3660
Epoch 3/30
332/332 [==============================] - 199s 599ms/step - loss: 1.4564 - rpn_class_loss: 0.0987 - rpn_bbox_loss: 0.4751 - mrcnn_class_loss: 0.1368 - mrcnn_bbox_loss: 0.3560 - mrcnn_mask_loss: 0.3898 - val_loss: 1.2970 - val_rp